<a href="https://colab.research.google.com/github/AmirAliShar/08_RAG/blob/main/04_VectorStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the libraries

In [5]:
#!pip install langchain-community qdrant_client langchain_huggingface qdrant_client pypdf openai langchain_qdrant

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
import openai
import os
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

# Loading data

In [8]:
loader = PyPDFLoader("National AI Policy Consultation Draft V1.pdf")
pages =loader.load()

In [9]:
len(pages)

41

In [10]:
pages[9]

Document(metadata={'source': 'National AI Policy Consultation Draft V1.pdf', 'page': 9}, page_content=' \n \n10 \n \n                           \nS.No. Pillar Target Description Current State Desired State Timeline \n1.  \nEnabling AI \nthrough \nAwareness & \nReadiness \nTarget-1 \n \nPublic Awareness \nof AI & Allied \nTechnologies \n \nThe level of awareness regarding AI and its \npotential applications for socio-economic \ndevelopment by addressing societal \nchallenges is considerably low.  \n \nA comprehensive program regarding public \nawareness of AI & allied technologies shall be \nundertaken at the national level to achieve a \nwidespread awareness of 90%  of the public \nwith internet access. This program under CoE-\nAI shall undertake curriculum revision at all \nlevels of education, training of the existing \nworkforce, and specialized programs for \nmarginalized women and PWDs.  \nBy 2026 \n \nTarget-2 \nDeveloping new \nskilled human \ncapital and \nupskilling the \nexis

# Preprocessing and Spelliting  Meta data

In [11]:
# Call the splitter
splitter =RecursiveCharacterTextSplitter(
    chunk_size =1500,
    chunk_overlap =150
    )

#Create an empty list to store processes documnect chunks
doc_list = []
# Iterate over each page in the extracted pages
for page in pages:
  #split the page context into smaller chunks
  chunk= splitter.split_text(page.page_content)

  #Iterate over each chunks and create doc object
  for chunks in chunk:
    #Metadata for each chunk ,including source and page number
    metadata = {"source":"AI Policy", "page_number":page.metadata['page'] +1}

    #Create a doc object with content and metadata
    doc_string = Document(page_content=chunks,metadata=metadata)

    #Apppend the doc obj to the list
    doc_list.append(doc_string)



In [12]:
# Check the length of  data
len(doc_list)

102

In [13]:
from langchain.docstore.document import Document

In [ ]:
# Print the 50 chunks data
doc_list[50]

Document(metadata={'source': 'AI Policy', 'page_number': 22}, page_content='7 \n \nAI can play a critical role in reducing road accidents by analyzing the crash data available, providing insights \nto avert collisions, and enhancing the response after the accidents. Besides autonomous cars that are now \na reality, AI applications fuel lane-keeping systems, emergency brakes, and auto-parking systems. \n4.1.4 Data and Computational Infrastructure \nThis initiative includes standardizing the data of  State-Owned Enterprises (SoEs), Boards , and Civil \nAuthorities for applying artificial Intelligence based Algorithms ensuring seamless service provisioning and \nanalytics. The AI Directorate and CoE-AI, through this policy intervention , are tasked to carry out the \nfollowing functions;  \nI. In consultation with stakeholders, the AI Regulatory Directorate (ARD) shall direct identified public \nsector entities and Authorities to standardize their data for efficient and secure use.  \nII.

# Embeddings and  Vector Store

In [14]:
# Create object of the embedding model
from langchain_huggingface import HuggingFaceEmbeddings
embed_model_HF = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Qdrant Vector Store

In [ ]:
qdrant_url ="YOUR_API_URL"
qdrant_key ="qezyD2PYrdXmTElc6VEhYttFbiTEEN5KiU_d5TYw"
collection_name ="AI_Policy"

In [ ]:
 # Intialize Qdrant VS with documents and embedding model

 qdrant = QdrantVectorStore.from_documents(
     doc_list,      #List of document objects to be stored in the vectore store
     embed_model_HF,
     url = qdrant_url,
     api_key =qdrant_key,
     collection_name = collection_name # Name of the collection to store the vector in
 )

# Query vector store

In [ ]:
Query ="What  is artificial intelligence policy for teachers"

In [ ]:
# Retrieve relavant documents
result = qdrant.similarity_search(Query,k=5)
# k show the how many vectors returns
result[3]

Document(metadata={'source': 'AI Policy', 'page_number': 5, '_id': '49daf0b4-d0d0-4a82-bd11-17a782b9a4c9', '_collection_name': 'AI_Policy'}, page_content='5 \n \nForeword \nThrough its continued efforts toward the early realization of the Digital Pakistan vision of the Government \nof Pakistan, the Ministry of IT & Telecom is committed to providing its people with timely and equal access \nto opportunities by stimulating a culture of innovation through an overarching de velopmental agenda \norchestrated to embrace cutting -edge technologies such as Artificial Intelligence efficiently and \nresponsibly.   \nIn this regard, the Artificial Intelligence (AI) Policy is a pivotal milestone for transforming Pakistan into a \nknowledge-based economy as it spells out a national strategy to establish an ecosystem necessary for AI \nadoption by harnessing an agile framework for addressing different aspects of unique user journeys \nencompassing different market horizontals and industry verticals 

In [ ]:
result[0].page_content

'4.2.3 Algorithms, Data Science & AI in Basic Education \nI. Where the policy document emphasizes the fundamental understanding and awareness of \npersonal data protection and AI, it also aims to stimulate an incremental impact of AI on society \nright from the grassroots. Therefore, the policy has given equal importance to teaching algorithms, \ndata science, and AI in basic STEM education. In this regard, it stipulates that. CoE-AI shall hire a \nlocal/international consultant with expertise in high -tech curriculum development to develop a \nNational Curriculum in Algorithms, Data Sciences, AI, and Allied Technologies from the sixth to the \ntwelfth standard.'

## Pinecone Vector store

In [18]:
from langchain_pinecone import PineconeVectorStore
import os
#Create the api key and index name on the pinecone
PINECONE_API_KEY = 'API_KEY_PINECONE'
index_name = "llm"

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [19]:
# Convert the document into vectors

vector = PineconeVectorStore.from_documents(
    doc_list,
    embed_model_HF,
    index_name=index_name
)

In [20]:
query = "What is artificial intelligence policy for teachers"
pinecone_result = vector.similarity_search(query, k=5)

In [21]:
pinecone_result

[Document(id='be19e324-8258-482d-9f14-c186ff43677f', metadata={'page': 25.0, 'source': 'National AI polacy'}, page_content='4.2.3 Algorithms, Data Science & AI in Basic Education \nI. Where the policy document emphasizes the fundamental understanding and awareness of \npersonal data protection and AI, it also aims to stimulate an incremental impact of AI on society \nright from the grassroots. Therefore, the policy has given equal importance to teaching algorithms, \ndata science, and AI in basic STEM education. In this regard, it stipulates that. CoE-AI shall hire a \nlocal/international consultant with expertise in high -tech curriculum development to develop a \nNational Curriculum in Algorithms, Data Sciences, AI, and Allied Technologies from the sixth to the \ntwelfth standard.'),
 Document(id='c02656ba-32af-47d9-b221-52442a9e3ac9', metadata={'page': 5.0, 'source': 'National AI polacy'}, page_content='5 \n \nForeword \nThrough its continued efforts toward the early realization of 

In [23]:
res= vector.similarity_search_with_score(query, k=1)


In [24]:
res

[(Document(id='ea21c84e-6ae5-46aa-b9b7-13fe5d397f7a', metadata={'page_number': 25.0, 'source': 'AI Policy'}, page_content='4.2.3 Algorithms, Data Science & AI in Basic Education \nI. Where the policy document emphasizes the fundamental understanding and awareness of \npersonal data protection and AI, it also aims to stimulate an incremental impact of AI on society \nright from the grassroots. Therefore, the policy has given equal importance to teaching algorithms, \ndata science, and AI in basic STEM education. In this regard, it stipulates that. CoE-AI shall hire a \nlocal/international consultant with expertise in high -tech curriculum development to develop a \nNational Curriculum in Algorithms, Data Sciences, AI, and Allied Technologies from the sixth to the \ntwelfth standard.'),
  0.798111737)]